In [ ]:
import os
import sys
import zipfile
from pathlib import Path

import basedosdados as bd
import pandas as pd
import requests

ROOT = Path("models") / "br_inep_indicadores_educacionais"

# Adiciona a pasta 'code' ao sys.path para permitir a importação de constants
CODE_DIR = ROOT / "code"
sys.path.append(str(CODE_DIR.resolve()))

from constants import (  # noqa: E402
    rename_afd,
    rename_atu,
    rename_dsu,
    rename_had,
    rename_icg,
    rename_ied,
    rename_ird,
    rename_tdi,
    rename_tnr,
    rename_tx,
)

# Diretórios de entrada e saída
INPUT_BR = ROOT / "tmp"
INPUT_BR.mkdir(exist_ok=True)

OUTPUT = ROOT / "output"
OUTPUT.mkdir(exist_ok=True)

URLS = [
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/ATU_2023_BRASIL_REGIOES_UF.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/tx_rend_brasil_regioes_ufs_2023.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/HAD_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/TDI_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/tnr_brasil_regioes_ufs_2024.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/DSU_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/AFD_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/IRD_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/IED_2023_BRASIL_REGIOES_UFS.zip",
    "https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2023/ICG_2023_BRASIL_REGIOES_UFS.zip",
]

In [ ]:
for url in URLS:
    filename = url.split("/")[-1]  # pega só o nome do arquivo
    filepath = os.path.join(INPUT_BR, filename)

    try:
        r = requests.get(url, stream=True, timeout=60)
        r.raise_for_status()  # erro se não for 200
        with open(filepath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Baixado: {filename}")
    except Exception as e:
        print(f"Erro ao baixar {url}: {e}")

In [ ]:
for file in os.listdir(INPUT_BR):
    path = os.path.join(INPUT_BR, file)
    if os.path.isfile(path) and file.lower().endswith(".zip"):
        try:
            with zipfile.ZipFile(path) as z:
                z.extractall(INPUT_BR)
            print(f"Extraído: {file}")
        except zipfile.BadZipFile:
            print(f"Arquivo inválido (não é zip de verdade): {file}")

In [ ]:
afd = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "AFD_2023_BRASIL_REGIOES_UF",
        "AFD_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=10,
)


afd = afd.rename(columns=rename_afd, errors="raise")

afd = afd.loc[afd["ano"] == 2023,]
afd["localizacao"] = afd["localizacao"].str.lower()
afd["rede"] = afd["rede"].str.lower().replace("pública", "publica")

## Média de alunos por turma (ATU)

atu = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "ATU_2023_BRASIL_REGIOES_UFS",
        "ATU_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=8,
)

atu = atu.rename(columns=rename_atu, errors="raise")

atu = atu.loc[atu["ano"] == 2023,]
atu["localizacao"] = atu["localizacao"].str.lower()
atu["rede"] = atu["rede"].str.lower().replace("pública", "publica")

# Percentual de Docentes com Curso Superior
dsu = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "DSU_2023_BRASIL_REGIOES_UFS",
        "DSU_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=9,
)


dsu = dsu.rename(columns=rename_dsu, errors="raise")

dsu = dsu.loc[dsu["ano"] == 2023,]
dsu["localizacao"] = dsu["localizacao"].str.lower()
dsu["rede"] = dsu["rede"].str.lower().replace("pública", "publica")


# Média de Horas-aula diária HAD -> 2023

had = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "HAD_2023_BRASIL_REGIOES_UFS",
        "HAD_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=8,
)


had = had.rename(columns=rename_had, errors="raise")

had = had.loc[had["ano"] == 2023,]
had["localizacao"] = had["localizacao"].str.lower()
had["rede"] = had["rede"].str.lower().replace("pública", "publica")

# Complexidade de Gestão da Escola (ICG) -> 2023

icg = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "ICG_2023_BRASIL_REGIOES_UFS",
        "ICG_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=8,
)


icg = icg.rename(columns=rename_icg, errors="raise")

icg = icg.loc[icg["ano"] == 2023,]
icg["localizacao"] = icg["localizacao"].str.lower()
icg["rede"] = icg["rede"].str.lower().replace("pública", "publica")

# Esforço Docente (IED) -> 2023

ied = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "IED_2023_BRASIL_REGIOES_UFS",
        "IED_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=10,
)

ied = ied.rename(columns=rename_ied, errors="raise")

ied = ied.loc[ied["ano"] == 2023,]
ied["localizacao"] = ied["localizacao"].str.lower()
ied["rede"] = ied["rede"].str.lower().replace("pública", "publica")

# Regularidade do Corpo Docente (IRD) -> 2023

ird = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "IRD_2023_BRASIL_REGIOES_UFS",
        "IRD_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=9,
)


ird = ird.rename(columns=rename_ird, errors="raise")

ird = ird.loc[ird["ano"] == 2023,]
ird["localizacao"] = ird["localizacao"].str.lower()
ird["rede"] = ird["rede"].str.lower().replace("pública", "publica")

# Taxas de Distorção Idade-série (TDI) -> 2023

tdi = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "TDI_2023_BRASIL_REGIOES_UFS",
        "TDI_BRASIL_REGIOES_UFS_2023.xlsx",
    ),
    skiprows=8,
)


tdi = tdi.rename(columns=rename_tdi, errors="raise")

tdi = tdi.loc[tdi["ano"] == 2023,]
tdi["localizacao"] = tdi["localizacao"].str.lower()
tdi["rede"] = tdi["rede"].str.lower().replace("pública", "publica")

# Taxa de Não Resposta (tnr) -> 2023

tnr = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "tnr_brasil_regioes_ufs_2023",
        "tnr_brasil_regioes_ufs_2023.xlsx",
    ),
    skiprows=8,
)


tnr = tnr.rename(columns=rename_tnr, errors="raise")

tnr = tnr.loc[tnr["ano"] == 2023,]
tnr["localizacao"] = tnr["localizacao"].str.lower()
tnr["rede"] = tnr["rede"].str.lower().replace("pública", "publica")

# Taxa de aprovação, reprovação, abandono -> 2023

tx = pd.read_excel(
    os.path.join(
        INPUT_BR,
        "tx_rend_brasil_regioes_ufs_2023",
        "tx_rend_brasil_regioes_ufs_2023.xlsx",
    ),
    skiprows=8,
)

tx = tx.rename(columns=rename_tx, errors="raise")

tx = tx.loc[tx["ano"] == 2023,]
tx["localizacao"] = tx["localizacao"].str.lower()
tx["rede"] = tx["rede"].str.lower().replace("pública", "publica")

In [8]:
# Ler diretório de estados e regiões (para mapear UFs e regiões)
bd_dir = bd.read_sql(
    "SELECT * FROM `basedosdados.br_bd_diretorios_brasil.uf`",
    billing_project_id="basedosdados-dev",
)
regioes = bd_dir["regiao"].unique()
estados = bd_dir["nome"].unique()

Downloading: 100%|██████████|


In [9]:
# AFD
afd_brasil_2023 = afd.loc[afd["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

afd_ufs_2023 = (
    afd.loc[afd["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

afd_regioes_2023 = afd.loc[afd["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# ATU
atu_brasil_2023 = atu.loc[atu["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

atu_ufs_2023 = (
    atu.loc[atu["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

atu_regioes_2023 = atu.loc[afd["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# DSU
dsu_brasil_2023 = dsu.loc[atu["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

dsu_ufs_2023 = (
    dsu.loc[dsu["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

dsu_regioes_2023 = dsu.loc[dsu["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# HAD
had_brasil_2023 = had.loc[atu["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

had_ufs_2023 = (
    had.loc[had["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

had_regioes_2023 = had.loc[had["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# ICG
icg_brasil_2023 = icg.loc[icg["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

icg_ufs_2023 = (
    icg.loc[icg["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

icg_regioes_2023 = icg.loc[icg["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# IED
ied_brasil_2023 = ied.loc[ied["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

ied_ufs_2023 = (
    ied.loc[ied["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

ied_regioes_2023 = ied.loc[ied["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# IRD
ird_brasil_2023 = ird.loc[ird["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

ird_ufs_2023 = (
    ird.loc[ird["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

ird_regioes_2023 = ird.loc[ird["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# TDI
tdi_brasil_2023 = tdi.loc[tdi["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

tdi_ufs_2023 = (
    tdi.loc[tdi["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

tdi_regioes_2023 = tdi.loc[tdi["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# TNR
tnr_brasil_2023 = tnr.loc[tnr["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

tnr_ufs_2023 = (
    tnr.loc[tnr["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

tnr_regioes_2023 = tnr.loc[tnr["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# TX
tx_brasil_2023 = tx.loc[tx["UNIDGEO"] == "Brasil"].drop(columns=["UNIDGEO"])

tx_ufs_2023 = (
    tx.loc[tx["UNIDGEO"].isin(estados)]  # Filtra apenas os estados
    .merge(
        bd_dir[
            ["sigla", "nome"]
        ],  # Faz o merge com bd_dir pegando sigla e nome
        left_on="UNIDGEO",  # Usa UNIDGEO da tabela tx
        right_on="nome",
    )  # Compara com nome da tabela bd_dir
    .drop(columns=["UNIDGEO", "nome"])  # Remove as colunas de nome
    .rename(columns={"sigla": "sigla_uf"})  # Renomeia sigla para sigla_uf
)

tx_regioes_2023 = tx.loc[tx["UNIDGEO"].isin(regioes)].rename(
    columns={"UNIDGEO": "regiao"}
)

# Brasil

In [15]:
br = bd.read_sql(
    """
SELECT
  *
FROM
  `basedosdados.br_inep_indicadores_educacionais.brasil`
""",
    billing_project_id="basedosdados-dev",
)

Downloading: 100%|██████████|


In [10]:
chaves = ["ano", "rede", "localizacao"]
brasil_2023 = tdi_brasil_2023.copy()
brasil_2023 = pd.merge(brasil_2023, tx_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, afd_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, atu_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, dsu_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, had_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, icg_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, ied_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, tnr_brasil_2023, on=chaves, how="left")
brasil_2023 = pd.merge(brasil_2023, ird_brasil_2023, on=chaves, how="left")

In [11]:
brasil_2023

,ano,localizacao,rede,tdi_ef,tdi_ef_anos_iniciais,tdi_ef_anos_finais,tdi_ef_1_ano,tdi_ef_2_ano,tdi_ef_3_ano,tdi_ef_4_ano,...,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado,ird_baixa_regularidade,ird_media_baixa,ird_media_alta,ird_alta
0,2023,total,total,11.7,7.5,17,3.1,5.1,8.2,9.4,...,2.7,3,2.8,1.9,0.9,5.2,8.7,31.8,45.7,13.8
1,2023,urbana,total,10.9,7,15.6,3,5.1,7.9,8.7,...,2.7,3,2.8,1.9,0.8,5.2,6.6,32.2,49.4,11.8
2,2023,rural,total,17.7,10.7,28,3.5,5.3,10.6,13.8,...,2.3,2.7,2.3,1.6,1.5,4.2,13.5,30.8,37.1,18.6
3,2023,total,federal,10.6,4.3,13.1,1.4,3.4,5.5,4.3,...,1.7,2.6,1.7,1.1,0.3,1,0.6,7.9,75.7,15.8
4,2023,urbana,federal,10.6,4.3,13.1,1.4,3.4,5.5,4.3,...,1.7,2.6,1.7,1.1,0.4,0.8,0.7,7.9,75.3,16.1
5,2023,rural,federal,--,--,--,--,--,--,--,...,1.7,2.5,1.4,0.7,0.2,2.3,0.0,7.8,77.8,14.4
6,2023,total,estadual,13.3,6.8,15.6,3,5.1,7.4,7.6,...,2.7,3,2.8,1.8,1.3,5,7.2,38.4,45.3,9.1
7,2023,urbana,estadual,12.8,6,15.1,2.6,4.5,6.7,6.7,...,2.7,3,2.8,1.8,1.2,5,6.5,38.7,47.3,7.5
8,2023,rural,estadual,22.2,16.6,25.1,8.3,12.1,16.1,19.9,...,2.2,2.5,2.3,1.5,1.9,5,10.7,37.1,35.9,16.3
9,2023,total,municipal,13.3,8.4,22.6,2.9,5.3,9.2,10.9,...,2,2.4,2,1.6,0.1,0,9.4,31.6,44.8,14.2


In [17]:
brasil_2023 = brasil_2023.replace("--", None)
brasil_2023

,ano,localizacao,rede,tdi_ef,tdi_ef_anos_iniciais,tdi_ef_anos_finais,tdi_ef_1_ano,tdi_ef_2_ano,tdi_ef_3_ano,tdi_ef_4_ano,...,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado,ird_baixa_regularidade,ird_media_baixa,ird_media_alta,ird_alta
0,2023,total,total,11.7,7.5,17,3.1,5.1,8.2,9.4,...,2.7,3,2.8,1.9,0.9,5.2,8.7,31.8,45.7,13.8
1,2023,urbana,total,10.9,7,15.6,3,5.1,7.9,8.7,...,2.7,3,2.8,1.9,0.8,5.2,6.6,32.2,49.4,11.8
2,2023,rural,total,17.7,10.7,28,3.5,5.3,10.6,13.8,...,2.3,2.7,2.3,1.6,1.5,4.2,13.5,30.8,37.1,18.6
3,2023,total,federal,10.6,4.3,13.1,1.4,3.4,5.5,4.3,...,1.7,2.6,1.7,1.1,0.3,1,0.6,7.9,75.7,15.8
4,2023,urbana,federal,10.6,4.3,13.1,1.4,3.4,5.5,4.3,...,1.7,2.6,1.7,1.1,0.4,0.8,0.7,7.9,75.3,16.1
5,2023,rural,federal,None,None,None,None,None,None,None,...,1.7,2.5,1.4,0.7,0.2,2.3,0.0,7.8,77.8,14.4
6,2023,total,estadual,13.3,6.8,15.6,3,5.1,7.4,7.6,...,2.7,3,2.8,1.8,1.3,5,7.2,38.4,45.3,9.1
7,2023,urbana,estadual,12.8,6,15.1,2.6,4.5,6.7,6.7,...,2.7,3,2.8,1.8,1.2,5,6.5,38.7,47.3,7.5
8,2023,rural,estadual,22.2,16.6,25.1,8.3,12.1,16.1,19.9,...,2.2,2.5,2.3,1.5,1.9,5,10.7,37.1,35.9,16.3
9,2023,total,municipal,13.3,8.4,22.6,2.9,5.3,9.2,10.9,...,2,2.4,2,1.6,0.1,0,9.4,31.6,44.8,14.2


In [ ]:
brasil_output_path = os.path.join(OUTPUT, "brasil.csv")
brasil_2023 = brasil_2023.drop("ano", axis=1)

brasil_2023.to_csv(brasil_output_path, index=False)

In [20]:
tb_brasil = bd.Table(
    dataset_id="br_inep_indicadores_educacionais", table_id="brasil"
)

tb_brasil.create(
    path=OUTPUT,  # Caminho para o arquivo csv ou parquet
    if_storage_data_exists="replace",
    if_table_exists="replace",
    source_format="csv",
)

Uploading files: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
2025-09-11 16:25:09.302 | WARNING  | basedosdados.upload.table:_load_staging_schema_from_data:105 -  Table brasil allready exists, replacing schema!
2025-09-11 16:25:10.385 | INFO     | basedosdados.upload.table:delete:809 -  Table brasil_staging was deleted!
2025-09-11 16:25:11.211 | SUCCESS  | basedosdados.upload.table:create:690 - Table brasil was created in staging!


# Região

In [21]:
regiao = bd.read_sql(
    """
SELECT
  *
FROM
  `basedosdados.br_inep_indicadores_educacionais.regiao`
""",
    billing_project_id="basedosdados-dev",
)

Downloading: 100%|██████████|


In [22]:
# Juntando todos os indicadores em uma só tabela
chaves = ["ano", "rede", "regiao", "localizacao"]
regioes = tdi_regioes_2023.copy()
regioes = pd.merge(regioes, tx_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, afd_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, atu_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, dsu_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, had_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, icg_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, ied_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, tnr_regioes_2023, on=chaves, how="left")
regioes = pd.merge(regioes, ird_regioes_2023, on=chaves, how="left")

In [23]:
regioes = regioes.replace("--", None)
regioes

,ano,regiao,localizacao,rede,tdi_ef,tdi_ef_anos_iniciais,tdi_ef_anos_finais,tdi_ef_1_ano,tdi_ef_2_ano,tdi_ef_3_ano,...,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado,ird_baixa_regularidade,ird_media_baixa,ird_media_alta,ird_alta
0,2023,Norte,total,total,18.9,12.5,26.7,5.2,7.7,12.8,...,2.1,2.2,1.9,2.1,1.7,3.9,12.7,28.4,42.5,16.4
1,2023,Norte,urbana,total,15.5,10.5,21.6,4.8,7.1,11.2,...,2.2,2.3,2.1,2.2,1.8,3.5,4.6,26.1,56.4,12.9
2,2023,Norte,rural,total,27.8,17.7,41.1,6.1,9.2,16.8,...,1.4,1.5,1.3,1.2,0.9,4,17.8,29.9,33.7,18.6
3,2023,Norte,total,federal,6.2,2.4,7.5,0,1.9,3.5,...,1.3,2.1,1.1,0.5,0,None,1.3,14.1,76.9,7.7
4,2023,Norte,urbana,federal,6.2,2.4,7.5,0,1.9,3.5,...,1.3,2.1,1.1,0.5,0,None,1.5,14.7,75.0,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023,Centro-Oeste,urbana,privada,3.2,3,3.5,3.4,2.3,3.5,...,3.6,4.1,3.6,3.1,None,3.8,7.9,29.8,51.3,11.0
86,2023,Centro-Oeste,rural,privada,7.7,2.1,11.1,0,1.3,0.7,...,2.7,3.1,2.2,2.6,None,None,5.0,45.0,35.0,15.0
87,2023,Centro-Oeste,total,publica,12,8.4,16.4,2.9,5.9,9.8,...,4.9,5.3,5.2,3.9,0.3,8,9.4,45.3,39.4,5.9
88,2023,Centro-Oeste,urbana,publica,11.6,8.1,15.9,2.7,5.7,9.6,...,5,5.4,5.2,3.9,0.3,10.6,9.5,46.7,39.1,4.7


In [24]:
regioes = regioes.drop("ano", axis=1)
regioes.to_csv(os.path.join(OUTPUT, "data.csv"), index=False)

In [25]:
tb_regiao = bd.Table(
    dataset_id="br_inep_indicadores_educacionais", table_id="regiao"
)

tb_regiao.create(
    path=OUTPUT,  # Caminho para o arquivo csv ou parquet
    if_storage_data_exists="replace",
    if_table_exists="replace",
    source_format="csv",
)

Uploading files: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]
2025-09-11 16:29:18.660 | WARNING  | basedosdados.upload.table:_load_staging_schema_from_data:105 -  Table regiao allready exists, replacing schema!
2025-09-11 16:29:19.661 | INFO     | basedosdados.upload.table:delete:809 -  Table regiao_staging was deleted!
2025-09-11 16:29:20.424 | SUCCESS  | basedosdados.upload.table:create:690 - Table regiao was created in staging!


# UF

In [26]:
ufs = bd.read_sql(
    """
SELECT
  *
FROM
  `basedosdados.br_inep_indicadores_educacionais.uf`
""",
    billing_project_id="basedosdados-dev",
)

Downloading: 100%|██████████|


In [27]:
# Juntando todos os indicadores em uma só tabela
chaves = ["ano", "rede", "sigla_uf", "localizacao"]
uf = tdi_ufs_2023.copy()
uf = pd.merge(uf, tx_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, afd_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, atu_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, dsu_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, had_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, icg_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, ied_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, tnr_ufs_2023, on=chaves, how="left")
uf = pd.merge(uf, ird_ufs_2023, on=chaves, how="left")

In [28]:
uf

,ano,localizacao,rede,tdi_ef,tdi_ef_anos_iniciais,tdi_ef_anos_finais,tdi_ef_1_ano,tdi_ef_2_ano,tdi_ef_3_ano,tdi_ef_4_ano,...,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado,ird_baixa_regularidade,ird_media_baixa,ird_media_alta,ird_alta
0,2023,total,total,10.1,5.8,14.7,2.4,4.4,7,6.4,...,3.6,3.3,3.8,3.7,1.3,2,4.0,24.2,54.8,17.0
1,2023,urbana,total,9.1,5,13.3,2.2,4.2,5.9,5.5,...,3.6,3.3,3.8,3.8,--,2,3.0,25.2,59.3,12.5
2,2023,rural,total,15.9,9.6,24.3,3.1,5.5,12.2,12,...,3.2,3.4,4.1,2,1.3,--,6.1,22.0,44.9,27.0
3,2023,total,federal,--,--,--,--,--,--,--,...,0.7,0.9,0.7,0.3,--,--,0.0,22.2,77.8,0.0
4,2023,urbana,federal,--,--,--,--,--,--,--,...,0.3,0.4,0.4,0.1,--,--,0.0,28.6,71.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2023,urbana,privada,2.7,2.1,3.4,2.2,1.6,2.4,2.3,...,5.1,6.1,4.6,4.2,--,9.5,10.6,33.3,45.8,10.3
475,2023,rural,privada,15.2,0,16.2,0,0,0,0,...,0.2,0.8,0,0,--,--,0.0,66.7,33.3,0.0
476,2023,total,publica,14.4,9.9,19.9,3,3.9,14.1,12.6,...,3.5,3.3,3.8,3.3,--,--,18.2,73.1,8.5,0.2
477,2023,urbana,publica,14.3,9.9,19.7,2.9,3.9,14.1,12.5,...,3.5,3.3,3.7,3.3,--,--,18.2,72.8,8.8,0.2


In [29]:
uf = uf.replace("--", None)
uf

,ano,localizacao,rede,tdi_ef,tdi_ef_anos_iniciais,tdi_ef_anos_finais,tdi_ef_1_ano,tdi_ef_2_ano,tdi_ef_3_ano,tdi_ef_4_ano,...,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado,ird_baixa_regularidade,ird_media_baixa,ird_media_alta,ird_alta
0,2023,total,total,10.1,5.8,14.7,2.4,4.4,7,6.4,...,3.6,3.3,3.8,3.7,1.3,2,4.0,24.2,54.8,17.0
1,2023,urbana,total,9.1,5,13.3,2.2,4.2,5.9,5.5,...,3.6,3.3,3.8,3.8,None,2,3.0,25.2,59.3,12.5
2,2023,rural,total,15.9,9.6,24.3,3.1,5.5,12.2,12,...,3.2,3.4,4.1,2,1.3,None,6.1,22.0,44.9,27.0
3,2023,total,federal,None,None,None,None,None,None,None,...,0.7,0.9,0.7,0.3,None,None,0.0,22.2,77.8,0.0
4,2023,urbana,federal,None,None,None,None,None,None,None,...,0.3,0.4,0.4,0.1,None,None,0.0,28.6,71.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2023,urbana,privada,2.7,2.1,3.4,2.2,1.6,2.4,2.3,...,5.1,6.1,4.6,4.2,None,9.5,10.6,33.3,45.8,10.3
475,2023,rural,privada,15.2,0,16.2,0,0,0,0,...,0.2,0.8,0,0,None,None,0.0,66.7,33.3,0.0
476,2023,total,publica,14.4,9.9,19.9,3,3.9,14.1,12.6,...,3.5,3.3,3.8,3.3,None,None,18.2,73.1,8.5,0.2
477,2023,urbana,publica,14.3,9.9,19.7,2.9,3.9,14.1,12.5,...,3.5,3.3,3.7,3.3,None,None,18.2,72.8,8.8,0.2


In [39]:
def salvar_particionado(df, output_path, particoes, nome_arquivo="uf.csv"):
    """
    Salva um DataFrame particionado no formato Hive (col=valor/col=valor/...).

    Args:
        df (pd.DataFrame): dataframe de entrada
        output_path (str): diretório raiz onde salvar
        particoes (list[str]): lista das colunas que definem partições
        nome_arquivo (str): nome do CSV dentro de cada partição (padrão: data.csv)
    """
    for keys, grupo in df.groupby(particoes):
        # garante que keys é sempre tupla
        if not isinstance(keys, tuple):
            keys = (keys,)

        # monta os pedaços da partição (ex: ano=2023/sigla_uf=SP)
        path_parts = [f"{col}={val}" for col, val in zip(particoes, keys)]
        path = os.path.join(output_path, *path_parts)
        os.makedirs(path, exist_ok=True)

        # salva sem as colunas de partição
        grupo.drop(columns=particoes).to_csv(
            os.path.join(path, nome_arquivo), index=False
        )

In [41]:
salvar_particionado(uf, OUTPUT, particoes=["ano", "sigla_uf"])

In [43]:
tb_uf = bd.Table(dataset_id="br_inep_indicadores_educacionais", table_id="uf")

tb_uf.create(
    path=OUTPUT,  # Caminho para o arquivo csv ou parquet
    if_storage_data_exists="replace",
    if_table_exists="replace",
    source_format="csv",
)

Uploading files: 100%|██████████| 27/27 [00:21<00:00,  1.28it/s]
2025-09-11 17:23:22.123 | SUCCESS  | basedosdados.upload.table:create:690 - Table uf was created in staging!


In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("tmp/brasil_uf_regiao/output/uf/ano=2024/uf=AC/uf.csv")

In [5]:
df.head()

,localizacao,rede,afd_ei_grupo_1,afd_ei_grupo_2,afd_ei_grupo_3,afd_ei_grupo_4,afd_ei_grupo_5,afd_ef_grupo_1,afd_ef_grupo_2,afd_ef_grupo_3,...,tnr_ef_6_ano,tnr_ef_7_ano,tnr_ef_8_ano,tnr_ef_9_ano,tnr_em,tnr_em_1_ano,tnr_em_2_ano,tnr_em_3_ano,tnr_em_4_ano,tnr_em_nao_seriado
0,rural,estadual,2.9,0.0,2.9,0.0,94.2,24.5,0.2,41.6,...,2.7,2.8,3.0,3.6,3.4,3.3,4.1,2.8,NaN,2.8
1,rural,federal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.9,1.4,1.0,0.0,NaN,NaN
2,rural,municipal,31.3,0.0,4.3,0.0,64.4,20.6,0.1,36.3,...,3.4,3.9,4.5,5.6,NaN,NaN,NaN,NaN,NaN,NaN
3,rural,publica,29.2,0.0,4.2,0.0,66.6,22.8,0.2,39.2,...,2.9,3.1,3.3,4.1,3.2,3.2,3.9,2.6,NaN,2.8
4,rural,total,29.2,0.0,4.2,0.0,66.6,22.8,0.2,39.2,...,2.9,3.1,3.3,4.1,3.2,3.2,3.9,2.6,NaN,2.8
